In [86]:
import pandas as pd
import re

In [96]:
#Carga de datos CSV 
data = pd.read_csv("L_STMaestroPrd.csv")
data.columns[data.isna().all()]
data_MasterProductos = data.dropna(axis=1, how='all')
data_MasterProductos["descrip1_descrip2"] = data_MasterProductos["descrip1"].fillna('') + " " + data_MasterProductos["descrip2"].fillna('')

C:\Users\OPERADOR\AppData\Local\Temp\ipykernel_8176\2563690654.py:2: DtypeWarning: Columns (7,14,17,18,21,28,39,42,46) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("L_STMaestroPrd.csv")
C:\Users\OPERADOR\AppData\Local\Temp\ipykernel_8176\2563690654.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_MasterProductos["descrip1_descrip2"] = data_MasterProductos["descrip1"].fillna('') + " " + data_MasterProductos["descrip2"].fillna('')


In [97]:
import pandas as pd
import re

def procesar_descripcion(data, nombre_columna):
    # Función para normalizar la descripción
    def normalizar(texto):
        # Asegurar que el texto sea un string
        texto = str(texto).strip().lower()
        
        # 1. Eliminar espacios redundantes
        texto = re.sub(r'\s+', ' ', texto)

        # 2. Remover caracteres especiales excepto letras, números, espacios y '%'
        texto = re.sub(r'[^a-zA-Z0-9\s%]', '', texto)

        # 3. Separar números de letras y letras de números (ej: 420mm -> 420 mm, mg250 -> mg 250)
        texto = re.sub(r'(\d)([a-zA-Z])', r'\1 \2', texto)
        texto = re.sub(r'([a-zA-Z])(\d)', r'\1 \2', texto)

        # 4. Manejo de 'x':
        #    a) Agregar espacios alrededor de la 'x' usada como separador (ej: 10x10 -> 10 x 10)
        texto = re.sub(r'(?<=\d)x(?=\d)', r' x ', texto)
        #    b) Separar 'x' de los números en casos como 'x15' o 'x4'
        texto = re.sub(r'\b[xX](\d+)', r'x \1', texto)

        # 5. Reemplazar separadores como guiones o barras con espacios
        texto = re.sub(r'[-/]', ' ', texto)

        # 6. Normalizar términos específicos combinados que terminan en 'x'
        combinaciones = {
            'tabx': 'tab x',
            'compx': 'comp x',
            'locx': 'loc x',
            'comprecx': 'comprec x',
            'capsx': 'caps x',
            'jabx': 'jab x',
            'pdax': 'pda x',
            'pvox': 'pvo x',
            'fcocvlvdosifx': 'fcocvlvdosif x',
            'sobx': 'sob x',
            'crx': 'cr x'
        }
        for key, val in combinaciones.items():
            texto = re.sub(rf'\b{key}\b', val, texto)

        # 7. Remover palabras repetidas consecutivas (ej: "paracetamol paracetamol")
        texto = ' '.join(sorted(set(texto.split()), key=texto.split().index))

        # 8. Manejo de unidades (ej: "10mg" y "10 mg" -> "10 mg")
        texto = re.sub(r'(\d+)\s*([a-zA-Z]+)', r'\1 \2', texto)

        # 9. Eliminar espacios redundantes nuevamente para garantizar limpieza
        texto = texto.strip()

        return texto

    # Aplicar la función 'normalizar' a cada elemento de la columna
    data_MasterProductos[nombre_columna] = data_MasterProductos[nombre_columna].apply(normalizar)
    
    return data


data_MasterProductos = procesar_descripcion(data_MasterProductos, "descrip1_descrip2")

C:\Users\OPERADOR\AppData\Local\Temp\ipykernel_8176\2465150903.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_MasterProductos[nombre_columna] = data_MasterProductos[nombre_columna].apply(normalizar)


In [89]:
import torch
from transformers import AutoTokenizer, BertModel


#Cargar tokenizador y modelo preentrenado
modelo_base = "dccuchile/bert-base-spanish-wwm-cased"
tokenizer = AutoTokenizer.from_pretrained(modelo_base)
model = BertModel.from_pretrained(modelo_base)


Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [98]:
#Tokenizamos cada descripcion
data_MasterProductos["token"] = data_MasterProductos["descrip1_descrip2"].apply(lambda x: tokenizer.tokenize(x))
data_MasterProductos["token"].iloc[3330:3351]

C:\Users\OPERADOR\AppData\Local\Temp\ipykernel_8176\801250840.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_MasterProductos["token"] = data_MasterProductos["descrip1_descrip2"].apply(lambda x: tokenizer.tokenize(x))


3330    [amo, ##xid, ##al, du, ##o, gt, ##s, ##x, 15, ml]
3331    [amo, ##xid, ##al, du, ##o, mu, ##c, compre, #...
3332    [amo, ##xid, ##al, du, ##o, sus, ##p, ##x, 120...
3333    [amo, ##xid, ##al, du, ##o, sus, ##p, ##x, 7, ...
3334             [amo, ##xid, ##al, gt, ##s, ##x, 15, ml]
3335    [amo, ##xid, ##al, ped, 125, mg, sus, ##p, ##x...
3336    [amo, ##xid, ##al, ped, 250, mg, sus, ##p, ##x...
3337    [amo, ##xid, ##al, ped, 250, mg, sus, ##p, ##x...
3338    [amo, ##xid, ##al, ped, 500, mg, sus, ##p, ##x...
3339    [amo, ##xid, ##al, ped, 500, mg, sus, ##p, ##x...
3340    [amo, ##xid, ##al, ped, 500, mg, sus, ##p, ##x...
3341    [amo, ##xid, ##al, respira, ##torio, 1, g, in,...
3342    [amo, ##xid, ##al, respira, ##torio, 1, g, in,...
3343    [amo, ##xid, ##al, respira, ##torio, cap, ##s,...
3344    [amo, ##xid, ##al, respira, ##torio, compre, #...
3345    [amo, ##xid, ##al, respira, ##torio, du, ##o, ...
3346    [amo, ##xid, ##al, respira, ##torio, du, ##o, ...
3347    [amo, 

In [93]:
from sklearn.metrics.pairwise import cosine_similarity

# Función para obtener el embedding de BERT
def obtener_embedding(texto):
    inputs = tokenizer(texto, return_tensors="pt", truncation=True, padding=True, max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()  # Promedio de las capas de BERT

#Crear pares de descripciones y calcular similitud 
pares = []
etiquetas = []



for _, fila_maestro in data_MasterProductos.iterrows():
    for _, fila_proveedor in df_proveedor.iterrows():
        desc_maestro = fila_maestro["descripcion_limpia"]
        desc_proveedor = fila_proveedor["descripcion_limpia"]

        # Obtener embeddings de las descripciones
        emb_maestro = obtener_embedding(desc_maestro)
        emb_proveedor = obtener_embedding(desc_proveedor)

        # Calcular la similitud coseno
        similitud = cosine_similarity([emb_maestro], [emb_proveedor])[0][0]

        # Definir el umbral de similitud
        umbral_similitud = 0.85

        # Si la similitud es mayor que el umbral, es una coincidencia
        if similitud >= umbral_similitud:
            etiquetas.append(1)  # Coincidencia positiva
        else:
            etiquetas.append(0)  # Coincidencia negativa

        # Guardar el par de descripciones
        pares.append((desc_maestro, desc_proveedor))


NameError: name 'df_maestro' is not defined

In [ ]:

# Crear el DataFrame de entrenamiento
df_entrenamiento = pd.DataFrame(pares, columns=["descripcion_maestro", "descripcion_proveedor"])
df_entrenamiento["etiqueta"] = etiquetas

# Guardar el dataset
df_entrenamiento.to_csv("dataset_entrenamiento.csv", index=False)